In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
from torchsummary import summary
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# import Gudhi Shape Dataset files
points = []
laplacians = []
vr_persistence_images = []
abstract_persistence_images = []

for i in range(0,10):
    points.append(np.genfromtxt('Gudhi Shape Dataset/shape_'+str(i)+'_points.csv', delimiter=',', skip_header=0))
    laplacians.append(np.genfromtxt('Gudhi Shape Dataset/shape_'+str(i)+'_laplacian.csv', delimiter=',', skip_header=0))
    vr_persistence_images.append(np.genfromtxt('Gudhi Shape Dataset/shape_'+str(i)+'_vr_persistence_image.csv', delimiter=',', skip_header=0))
    abstract_persistence_images.append(np.genfromtxt('Gudhi Shape Dataset/shape_'+str(i)+'_abstract_persistence_image.csv', delimiter=',', skip_header=0))


# import labels for the shapes
shape_labels = np.genfromtxt('Gudhi Shape Dataset/shape_labels.csv', delimiter=',', skip_header=1)
shape_labels = shape_labels.astype(int)[:,2]
print(shape_labels)

[1 1 1 1 1 0 0 0 0 0]


In [3]:
class ShapeDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = []
        self.labels = []

        for i in range(len(data)):
            for row in data[i]:
                self.data.append(torch.tensor(row, dtype=torch.float32))
                self.labels.append(labels[i])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [4]:
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4):
        super(DNN, self).__init__()
        
        # Fully connected layers
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = nn.Linear(hidden_dim4, 1)  # Output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))  # Apply ReLU after each hidden layer
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)  # Output layer without activation
        return torch.sigmoid(x)  # Apply sigmoid activation at the output layer

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cpu


In [6]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    tk0 = tqdm(train_loader, total=len(train_loader))
    train_loss = 0
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1), target.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        tk0.set_postfix(loss=loss.item())
    
    return train_loss / len(train_loader)

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output.view(-1), target.float()).item()
            pred = output.round()
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

In [7]:
def train_and_test_for_data_type(data_type, data, labels, input_dim):
    # Create dataset and split into train/test sets
    dataset = ShapeDataset(data, labels)
    train_data, test_data = train_test_split(dataset.data, test_size=0.2, random_state=42)
    train_labels, test_labels = train_test_split(dataset.labels, test_size=0.2, random_state=42)

    # Convert to custom Dataset format for train and test sets
    train_dataset = torch.utils.data.TensorDataset(torch.stack(train_data), torch.tensor(train_labels))
    test_dataset = torch.utils.data.TensorDataset(torch.stack(test_data), torch.tensor(test_labels))

    # Create DataLoaders
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Set hidden layer sizes
    hidden_dim1 = 512
    hidden_dim2 = 256
    hidden_dim3 = 128 
    hidden_dim4 = 64
    model = DNN(input_dim=input_dim, hidden_dim1=hidden_dim1, hidden_dim2=hidden_dim2,
                hidden_dim3=hidden_dim3, hidden_dim4=hidden_dim4).to(device)

    # Define optimizer and loss function
    learning_rate = 0.001
    momentum = 0.9
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    criterion = nn.BCELoss() # Binary Cross Entropy Loss (used for binary classification)

    epoch_results = []

    # Training loop
    num_epochs = 10
    for epoch in range(1, num_epochs + 1):
        print(f"Training model for {data_type} - Epoch {epoch}/{num_epochs}")
        train_loss = train(model, device, train_dataloader, optimizer, criterion, epoch)
        test_loss, accuracy = test(model, device, test_dataloader, criterion)
        
        # Store results
        epoch_results.append({
            'Epoch': epoch,
            'Train Loss': train_loss,
            'Test Loss': test_loss,
            'Test Accuracy (%)': accuracy
        })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    print(f"\nTraining and testing results for {data_type}:")
    print(epoch_results_df)

In [8]:
# Train and test for Laplacians (input_dim=1000)
train_and_test_for_data_type("Laplacians", laplacians, shape_labels, input_dim=1000)

# Train and test for VR Persistence Images (input_dim=100)
train_and_test_for_data_type("VR Persistence Images", vr_persistence_images, shape_labels, input_dim=100)

# Train and test for Abstract Persistence Images (input_dim=100)
train_and_test_for_data_type("Abstract Persistence Images", abstract_persistence_images, shape_labels, input_dim=100)

Training model for Laplacians - Epoch 1/10


  0%|          | 0/500 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 2/10


  0%|          | 0/500 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 3/10


...

Training model for Abstract Persistence Images - Epoch 9/10


  0%|          | 0/50 [00:00<?, ?it/s]

Training model for Abstract Persistence Images - Epoch 10/10


  0%|          | 0/50 [00:00<?, ?it/s]


Training and testing results for Abstract Persistence Images:
   Epoch  Train Loss  Test Loss  Test Accuracy (%)
0      1    0.654809   0.021061               57.5
1      2    0.622459   0.020366               60.5
2      3    0.609946   0.020158               77.0
3      4    0.601130   0.019867               76.5
4      5    0.598637   0.019773               77.0
5      6    0.590864   0.019809               76.5
6      7    0.587776   0.019419               76.0
7      8    0.582979   0.019646               77.0
8      9    0.581318   0.019513               77.0
9     10    0.578004   0.018779               76.5
